# Requierments


In [1]:
!pip install findspark
!pip install confluent-kafka
# Downloaded from https://mvnrepository.com/artifact/org.apache.spark/spark-streaming-kafka-0-8-assembly_2.11
!wget https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.0/spark-streaming-kafka-0-8-assembly_2.11-2.4.0.jar

--2020-04-23 17:07:50--  https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.0/spark-streaming-kafka-0-8-assembly_2.11-2.4.0.jar
Resolving repo1.maven.org (repo1.maven.org)... 151.101.112.209
Connecting to repo1.maven.org (repo1.maven.org)|151.101.112.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13385346 (13M) [application/java-archive]
Saving to: ‘spark-streaming-kafka-0-8-assembly_2.11-2.4.0.jar.2’

spark-streaming-kaf 100%[===================>]  12.76M  5.94MB/s    in 2.1s    

2020-04-23 17:07:52 (5.94 MB/s) - ‘spark-streaming-kafka-0-8-assembly_2.11-2.4.0.jar.2’ saved [13385346/13385346]



In [2]:
import os
import findspark
findspark.init('/usr/local/spark/spark-2.4.0-bin-hadoop2.7')
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars spark-streaming-kafka-0-8-assembly_2.11-2.4.0.jar pyspark-shell'

---

In [3]:
import pyspark
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext, Row
from pyspark.sql.types import StructType, StructField, DoubleType, StringType, IntegerType
from pyspark.sql.functions import udf
from pyspark.ml.feature import CountVectorizerModel, IDFModel, StandardScalerModel, Tokenizer
from pyspark.ml.classification import LogisticRegressionModel
from pyspark.streaming.kafka import KafkaUtils

from urllib.parse import unquote

APP_NAME = "BigData"
conf = pyspark.SparkConf().setAll([ ('spark.app.name', APP_NAME),
                                    ('spark.executor.memory', '8g'),
                                    ('spark.cores.max', '2'),
                                    ('spark.driver.memory','8g'),
                                   ('spark.master', 'local[2]')])
sc = SparkContext(conf=conf)
sqlc = SQLContext(sc)
sc

<SparkContext master=local[2] appName=BigData>

In [4]:
def to_ngram(payload_obj):
    n=2
    payload = str(payload_obj)
    ngrams = ''
    for i in range(0,len(payload)-n + 1):
        ngrams += payload[i:i+n]+ ' '
    return ngrams[:-1]

ngrams = udf(to_ngram, StringType())
tokenizer = Tokenizer.load('models/Tokenizer')
vectorizer = CountVectorizerModel.load('models/Vectorizer')
idf_model = IDFModel.load('models/idf')
scalerModel = StandardScalerModel.load('models/scalerModel')
model = LogisticRegressionModel.load('models/Logistic_Regression_Model')


topic = "test"
brokers = "localhost:9092"
ssc = StreamingContext(sc, batchDuration= 3)
kvs = KafkaUtils.createDirectStream(ssc, [topic], {"metadata.broker.list": brokers})


print("ready to run")

ready to run


In [5]:
def to_ngram(payload_obj):
    n=2
    payload = str(payload_obj)
    ngrams = []
    for i in range(0,len(payload)-n+1):
        ngrams.append(payload[i:i+n])
    return ngrams


def get_prediction(queries):

    try:
        queries = queries.map(lambda w: Row(payload=w))
        queries = sqlc.createDataFrame(queries)
        queries = queries.withColumn('ngrams', ngrams(queries['payload']))
        queries = tokenizer.transform(queries)
        queries = vectorizer.transform(queries)
        queries = idf_model.transform(queries)
        queries = scalerModel.transform(queries)
        preds = model.transform(queries)
        preds.select('payload','prediction').show()
    except : 
        print('No data')
    

In [6]:
queries = kvs.map(lambda x: unquote(x[1]))
# ngrams = queries.map(lambda x: to_ngram(x))
queries.foreachRDD(get_prediction)


ssc.start()
ssc.awaitTermination()  

No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
+--------------------+----------+
|             payload|prediction|
+--------------------+----------+
|/top.php?stuff='u...|       1.0|
|/h21y8w52.nsf?<sc...|       1.0|
|/ca000001.pl?acti...|       1.0|
|/scripts/edit_ima...|       1.0|
| /javascript/mta.exe|       1.0|
|/examples/jsp/col...|       1.0|
|/examples/jsp/cal...|       1.0|
|/phpwebfilemgr/in...|       1.0|
|/cgi-bin/script/c...|       1.0|
|/examples/jsp/cal...|       1.0|
+--------------------+----------+

+--------------------+----------+
|             payload|prediction|
+--------------------+----------+
|/moodle/filter/te...|       1.0|
|/examples/jsp/col...|       1.0|
|/id;1627282494;fp...|       1.0|
|/<img src="javasc...|       1.0|
|/bmeun223.exe?<me...|       1.0|
| /htbin/windmail.exe|       1.0|
|/<script>document...|       1.0|
|/javascript/.pass...|       1.0|
|/ope

+--------------------+----------+
|             payload|prediction|
+--------------------+----------+
|            /ka'roo/|       1.0|
|<script>document....|       1.0|
|/javascript/maest...|       1.0|
|/top.php?stuff='&...|       1.0|
|/cacti/setcookie....|       1.0|
|/javascript/vendo...|       1.0|
|   /scripts/tree.php|       1.0|
|  /subscriptions.yml|       0.0|
|/scripts/4rzefq7d...|       1.0|
|/help.php?q="\x0a...|       1.0|
|/pwcgi/smpwservic...|       1.0|
| /javascript/bfc.exe|       1.0|
|/cgi-bin/rss.php?...|       1.0|
|/cacti/include/do...|       1.0|
|   /snmx-cgi/fxm.exe|       1.0|
|      /descriptious/|       1.0|
|/main.php?stuff="...|       1.0|
|/manager/lang/ind...|       1.0|
|/cgi-bin/generate...|       1.0|
|/o35zot2r.cfm?<sc...|       1.0|
+--------------------+----------+

+--------------------+----------+
|             payload|prediction|
+--------------------+----------+
|/scripts/inc/pipe...|       1.0|
|/prescription-ult...|       0.0|
|/t7tm4m0b.cf

+--------------------+----------+
|             payload|prediction|
+--------------------+----------+
|/examples/jsp/num...|       1.0|
|/5qnlm5z2.dll?<me...|       1.0|
|/fcgi-bin/filesee...|       1.0|
|/scripts/kfzqb5x8...|       1.0|
|/b4vng02k.asp?<sc...|       1.0|
|/scripts/item.fts...|       1.0|
| /scripts/test11.asp|       1.0|
|/main.php?stuff="...|       1.0|
|/?<script>documen...|       1.0|
|/en-us/account/in...|       1.0|
+--------------------+----------+

+--------------------+----------+
|             payload|prediction|
+--------------------+----------+
|/cgi-915/pfdispla...|       1.0|
|/phpmyadmin/htsea...|       1.0|
|/fc9t54l7.jsp?<me...|       1.0|
|/forum1_professio...|       1.0|
|/admin/doeditconf...|       1.0|
|/examples/jsp/jsp...|       1.0|
|/javascript/benef...|       1.0|
|/hrttz9fj.exe?<me...|       1.0|
|/top.php?stuff=&v...|       1.0|
|/scripts/kernel/l...|       1.0|
|/h21y8w52.aspx?<i...|       1.0|
|ftp://192.168.24....|       1.0|
|/l752x1ry.ks

+--------------------+----------+
|             payload|prediction|
+--------------------+----------+
|/x7pclv4s.kspx?<m...|       1.0|
|/scripts/quicksto...|       1.0|
|/scripts/phprint....|       1.0|
|/fnwhrxyd.htm?<sc...|       1.0|
|/examples/jsp/ses...|       1.0|
|/sysuser/docmgr/m...|       1.0|
|/examples/servlet...|       1.0|
|/bxnyrhmh.aspx?<m...|       1.0|
|/scripts/misc/inf...|       1.0|
|/ybz5rz7a.html?<s...|       1.0|
+--------------------+----------+

+--------------------+----------+
|             payload|prediction|
+--------------------+----------+
|<script>document....|       1.0|
|/help.php?q="&pin...|       1.0|
|/<meta http-equiv...|       1.0|
|/main.php?stuff="...|       1.0|
|/scripts/ubbthrea...|       0.0|
|/azcrfs8d.jspa?<m...|       1.0|
|/examples/servlet...|       1.0|
|/b5xdqgz2.cgi?<sc...|       1.0|
|/169okeyj.mscgi?<...|       1.0|
|/scripts/axdgpyi2...|       1.0|
|/scripts/msmmask....|       1.0|
|/recordings/admin...|       1.0|
|/examples/js

+--------------------+----------+
|             payload|prediction|
+--------------------+----------+
|/j4rdyhw8.fts?<sc...|       1.0|
|/xoops232b/htdocs...|       1.0|
|/scripts/ylrvly9n...|       1.0|
|/scripts/index.ph...|       1.0|
|/scripts/vhost.ph...|       1.0|
|/scripts/kfzqb5x8...|       1.0|
|/newsletter_log.p...|       1.0|
|/ao22ww7y.do?<scr...|       1.0|
|/javascript/globa...|       1.0|
|/b5xdqgz2.exe?<sc...|       1.0|
|/jw64yq8u.x?<meta...|       1.0|
|  /scripts/gitweb.pl|       1.0|
|/main.php?logout=...|       1.0|
|/scripts/cal_cat....|       1.0|
|/uvazfs7p.pl?<met...|       1.0|
|/cgi-bin/insertor...|       1.0|
|/examples/jsp/err...|       1.0|
|/top.php?stuff='\...|       1.0|
|/manager/plog-dow...|       1.0|
|/javascript/subsc...|       1.0|
+--------------------+----------+

+--------------------+----------+
|             payload|prediction|
+--------------------+----------+
|/ju0u7kfo.asp?<im...|       1.0|
|/manager/admin/ad...|       1.0|
|/recordings/

KeyboardInterrupt: 